In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [12]:
scoring_df = pd.read_csv('scoring_dataset.csv')
defense_diff_df = pd.read_csv('defense_diff_dataset.csv')

scoring_columns_to_keep = [
    'PLAYER', 'GP', 'MIN', 'PTS', 'DRIVES',
    'C&S\nPTS', 'C&S\nFG%', 
    'Pull Up\nPTS', 'Pull Up\nFG%', 
    'eFG%', 'Shot Quality', 
    'Assisted 2s\n PTS', 'Unassisted 2s\n PTS',
    'Assisted 3s\n PTS', 'Unassisted 3s\n PTS'
]

defense_diff_columns_to_keep = [
    'PLAYER', 'DIFF%'
]

cleaned_scoring = scoring_df[scoring_columns_to_keep].copy()
cleaned_defense_diff = defense_diff_df[defense_diff_columns_to_keep].copy()

cleaned_scoring.columns = [
    'Player', 'GP', 'MIN', 'PPG', 'Drives',
    'C&S PTS', 'C&S FG%',
    'PU PTS', 'PU FG%',
    'eFG%', 'Shot Quality',
    'Assisted 2s', 'Unassisted 2s',
    'Assisted 3s', 'Unassisted 3s'
]

percent_cols = ['C&S FG%', 'eFG%', 'PU FG%']

for col in percent_cols:
    cleaned_scoring[col] = cleaned_scoring[col].astype(str).str.rstrip('%').astype(float) / 100

Unassisted_PTS = cleaned_scoring['Unassisted 2s'] + cleaned_scoring['Unassisted 3s']
cleaned_scoring['Unassisted PTS'] = Unassisted_PTS
cleaned_scoring['Unassisted%'] = cleaned_scoring['Unassisted PTS'] / cleaned_scoring['PPG']
cleaned_scoring['C&S%'] = cleaned_scoring['C&S PTS'] / cleaned_scoring['PPG']
cleaned_scoring['Gap'] = cleaned_scoring['eFG%'] - cleaned_scoring['Shot Quality']

cleaned_scoring
cleaned_defense_diff


,PLAYER,DIFF%
0,Jamal Murray,1.5
1,Austin Reaves,3.3
2,Josh Hart,2.1
3,Dyson Daniels,1.7
4,De'Aaron Fox,1.6
...,...,...
136,Cameron Payne,1.2
137,Seth Curry,2.5
138,Blake Wesley,3.8
139,Caleb Houstan,3.7


In [42]:
fig = px.scatter(
    cleaned,
    x='Shot Quality',
    y='eFG%',
    hover_name='Player',
    title='eFG% vs Shot Quality',
    labels={'Shot Quality': 'Shot Quality (Expected eFG%)', 'eFG%': 'Actual eFG%'},
    width=800,
    height=600
)

fig.add_trace(
    go.Scatter(
        x=[cleaned['Shot Quality'].min(), cleaned['Shot Quality'].max()],
        y=[cleaned['Shot Quality'].min(), cleaned['Shot Quality'].max()],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='actual = expected'
    )
)

fig.update_layout(template='plotly_white')
fig.show()


In [43]:
fig = px.scatter(
    cleaned,
    x='Shot Quality',
    y='Gap',
    hover_name='Player',
    title='Gap vs Shot Quality',
    labels={'Shot Quality': 'Shot Quality (Expected eFG%)', 'Gap': 'Gap (Actual eFG% - Expected eFG%)'},
    width=800,
    height=600
)

fig.add_trace(
    go.Scatter(
        x=[cleaned['Shot Quality'].min(), cleaned['Shot Quality'].max()],
        y=[0, 0],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='y = 0'
    )
)

fig.update_layout(template='plotly_white')
fig.show()

In [44]:
x = cleaned['Unassisted%']
y = cleaned['eFG%']

m, b = np.polyfit(x, y, 1)
line_x = np.linspace(x.min(), x.max(), 100)
line_y = m * line_x + b

fig = px.scatter(
    cleaned,
    x='Unassisted%',
    y='eFG%',
    hover_name='Player',
    title='Unassisted Scoring % vs eFG% for NBA Guards',
    labels={
        'Unassisted%': 'Unassisted Points (%)',
        'eFG%': 'Effective Field Goal %'
    },
    width=800,
    height=600
)

fig.add_trace(
    go.Scatter(
        x=line_x,
        y=line_y,
        mode='lines',
        name='Trendline',
        line=dict(color='red', dash='dash')
    )
)

fig.update_layout(template='plotly_white')
fig.show()


In [4]:
x = cleaned['C&S%']
y = cleaned['eFG%']

m, b = np.polyfit(x, y, 1)
line_x = np.linspace(x.min(), x.max(), 100)
line_y = m * line_x + b

fig = px.scatter(
    cleaned,
    x='C&S%',
    y='eFG%',
    hover_name='Player',
    title='Unassisted Scoring % vs eFG% for NBA Guards',
    labels={
        'C&S%': 'Catch and Shoot Points (%)',
        'eFG%': 'Effective Field Goal %'
    },
    width=800,
    height=600
)

fig.add_trace(
    go.Scatter(
        x=line_x,
        y=line_y,
        mode='lines',
        name='Trendline',
        line=dict(color='red', dash='dash')
    )
)

fig.update_layout(template='plotly_white')
fig.show()
